<a href="https://colab.research.google.com/github/rmoberly91/Sentiscope_Repo/blob/main/sentiscope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#utilities
import re
import numpy as np
import pandas as pd

#plotting
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

#preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_curve, auc


#nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

#sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

#warnings
import warnings
warnings.filterwarnings('ignore')

#Data import
#from google.colab import drive
#drive.mount('/content/drive')
#%cd /content/drive/My Drive/Colab Notebooks/


In [ ]:
#Import dataset

DATASET_COLUMNS  = ["sentiment", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
dataset = pd.read_csv("twitter_data140.csv", encoding=DATASET_ENCODING,
                      names=DATASET_COLUMNS, engine = "python")

#What we care about is the Sentiment, everything else is not super important
dataset = dataset[['sentiment','text']]

#Replacing "4" with "1" in the sentiment portion of the data
dataset['sentiment'] = dataset['sentiment'].replace(4,1)

#Visualization of the dataset
ax = dataset.groupby('sentiment').count().plot(kind='bar', title='Distribution of data', legend=False)
ax.set_xticklabels(['Negative','Positive'], rotation=0)

#Create lists for our dataset
text, sentiment = list(dataset['text']), list(dataset['sentiment'])

#Checking for unique values in the sentiment column
dataset['sentiment'].unique()

#Setting our positive and negative
data_pos = dataset[dataset['sentiment'] == 1]
data_neg = dataset[dataset['sentiment'] == 0]

#Combining positive and negative sets together for easy use
dataset = pd.concat([data_pos, data_neg])

#Forcing the text in the dataset to lowercase
dataset['text']=dataset['text'].str.lower()
dataset['text'].tail()

In [ ]:
#Remove Usernames
#Search for regular expressions with a @ at the beginning
#Then we just remove those regular expressions. Simple!

def remove_usernames(tweet):
    tweet = re.sub('@[^\s]+','',tweet)
    return tweet
dataset['text'] = dataset['text'].apply(lambda x: remove_usernames(x))
dataset['text'].tail()

In [ ]:
#Removing URLS from our dataset
def cleaning_URLs(data):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',data)
dataset['text'] = dataset['text'].apply(lambda x: cleaning_URLs(x))
dataset['text'].tail()

In [ ]:
#Removing numbers from the dataset
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
dataset['text'] = dataset['text'].apply(lambda x: cleaning_numbers(x))
dataset['text'].tail()

In [ ]:
#Stop Words detection
#Stop Words are words that are going to be irrelevant for our use
#So we just remove them here.

", ".join(stopwords.words('english'))

stopwords_list = stopwords.words('english')

STOPWORDS = set(stopwords.words('english'))
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

dataset['text'] = dataset['text'].apply(lambda text: cleaning_stopwords(text))
dataset['text'].head()

In [ ]:
#Removing Punctuation
#Punctuation creates a big problem, so we remove it

import string

english_punctuations = string.punctuation
punctuations_list = english_punctuations

def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

dataset['text']= dataset['text'].apply(lambda x: cleaning_punctuations(x))
dataset['text'].tail()

In [ ]:
#Stemmer - pulling words out by the stem
#The basic root of the word

st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data
dataset['text']= dataset['text'].apply(lambda x: stemming_on_text(x))
dataset['text'].head()

In [ ]:
#Lemmatizer - makes lemons
#Reduces words to their lemma, the essence of the word

lm = nltk.WordNetLemmatizer()
def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return data
dataset['text'] = dataset['text'].apply(lambda x: lemmatizer_on_text(x))
dataset['text'].head()

In [ ]:
#Splitting up data for validation and training
X_Train, X_Test, y_train, y_test = train_test_split(dataset['text'], dataset['sentiment'], test_size=0.25, random_state=651329)

In [ ]:
#Fitting the TFID Vectorizer
vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=500000)
vectoriser.fit(X_Train)
print('No. of feature_words: ', len(vectoriser.get_feature_names_out()))

In [ ]:
#Using vectorizer data
X_train = vectoriser.transform(X_Train)
X_test  = vectoriser.transform(X_Test)

In [ ]:
#Evaluate the model and see if it works!
def model_Evaluate(model):
    # Predict values for Test dataset
    y_pred = model.predict(X_test)
    # Print the evaluation metrics for the dataset.
    print(classification_report(y_test, y_pred))

    cf_matrix = confusion_matrix(y_test, y_pred)
    categories = ['Negative','Positive']
    group_names = ['True Neg','False Pos', 'False Neg','True Pos']
    group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]
    labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_names,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(cf_matrix, annot = labels, cmap = 'Blues',fmt = '',
    xticklabels = categories, yticklabels = categories)
    plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
    plt.ylabel("Actual values" , fontdict = {'size':14}, labelpad = 10)
    plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)

In [ ]:
#Boilerplate Logistic Regression
LRmodel = LogisticRegression(solver='lbfgs')
LRmodel.fit(X_train, y_train)
model_Evaluate(LRmodel)
y_pred = LRmodel.predict(X_test)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=1, label='ROC curve (area = %0.2f)' % roc_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC CURVE')
plt.legend(loc="lower right")
plt.show()